<a href="https://colab.research.google.com/github/GabrielBG0/Fakenews-Recognition/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Deleting default sample data folder
!rm -rf /content/sample_data
!rm -rf /content/FakeNewsNet
!wget https://github.com/GabrielBG0/Fakenews-Recognition/raw/main/FakeNewsNet.zip
!unzip FakeNewsNet.zip

--2021-05-14 02:29:55--  https://github.com/GabrielBG0/Fakenews-Recognition/raw/main/FakeNewsNet.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/GabrielBG0/Fakenews-Recognition/main/FakeNewsNet.zip [following]
--2021-05-14 02:29:55--  https://raw.githubusercontent.com/GabrielBG0/Fakenews-Recognition/main/FakeNewsNet.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28359746 (27M) [application/zip]
Saving to: ‘FakeNewsNet.zip.1’

FakeNewsNet.zip.1   100%[===================>]  27.05M  --.-KB/s    in 0.1s    

2021-05-14 02:29:55 (223 MB/s) - ‘FakeNewsNet.zip.1’ saved [28359746/28359746]

Ar

In [ ]:
# Install aditional librarys
!pip install transformers
!pip install datasets

In [ ]:
import pickle
import numpy as np


documents = pickle.load(open("FakeNewsNet/dict_documents.pkl", "rb"))
indexes = np.load('FakeNewsNet/indexes.npy')
lbls = np.load('FakeNewsNet/labels.npy')

texts = []
labels = []
for i in range(len(indexes)):
    index = indexes[i]
    texts.append(documents[index])
    if lbls[i] == np.str_("-1"):
      labels.append(0)
    else :
      labels.append(1)

print("lable 1: " + str(labels[0]))
print("text 1: " + texts[0])

lable 1: 0
text 1: When I revealed in this column last year that Dame Helen Mirren was the new face of L’Oreal, I had hundreds of letters from readers, ecstatic that beauty giants are celebrating older women.  As these exclusive pictures show, from next week (February 5) Helen will be appearing in her first television advertising campaign for the company.  The 69-year-old actress will be seen championing the role of the older woman — and the candid, unretouched shots from the advert show that wrinkles, ageing hands and natural make-up can be beautiful.  Scroll down for video  Dame Helen Mirren photographed for the L'Oreal Paris age perfect campaign. In this shot Helen wears the Age Perfect day cream, the candid, unretouched shots are a refreshing change and champion the beauty of the older woman  I suspect this approach will sell more products than the glossier, airbrushed campaigns that many women feel don’t relate to them and their real lives.  During the filming of the advert, Dame 

In [ ]:
count = 0
for text in texts:
  if len(text.split(' ')) > 512:
    count += 1
count

3816

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.2)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [ ]:
train_encodings = tokenizer(train_texts, padding=False)
val_encodings = tokenizer(val_texts, padding=False)
test_encodings = tokenizer(test_texts, padding=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (773 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
import copy
# CLS token id = 101, SEP token id = 102
def truncate(encodings, method="h", max_len=512):
  aux = copy.deepcopy(encodings)

  if method is "t":
    print('methond selected was: tail')
    #for encoding in aux["input_ids"]:
    for i in range(len(aux["input_ids"])):
      if len(aux["input_ids"][i]) > max_len:
        aux_input_ids = [101, *aux["input_ids"][i][-max_len+1:]]
        aux["input_ids"][i] = aux_input_ids.copy()
      else:
        aux["input_ids"][i] = [*aux["input_ids"][i], *[0]*(max_len - len(aux["input_ids"][i]))]
    for i in range(len(aux["token_type_ids"])):
      if len(aux["token_type_ids"][i]) > max_len:
        aux_token_type_ids = [aux["token_type_ids"][i][0], *aux["token_type_ids"][i][-max_len+1:]]
        aux["token_type_ids"][i] = aux_token_type_ids.copy()
      else:
        aux["token_type_ids"][i] = [*aux["token_type_ids"][i], *[0]*(max_len - len(aux["token_type_ids"][i]))]
    for i in range(len(aux["attention_mask"])):
      if len(aux["attention_mask"][i]) > max_len:
        aux_attention_mask = [aux["attention_mask"][i][0], *aux["attention_mask"][i][-max_len+1:]]
        aux["attention_mask"][i] = aux_attention_mask.copy()
      else:
        aux["attention_mask"][i] = [*aux["attention_mask"][i], *[0]*(max_len - len(aux["attention_mask"][i]))]

  elif method is "h":
    print('methond selected was: head')
    for i in range(len(aux["input_ids"])):
      if len(aux["input_ids"][i]) > max_len:
        aux_input_ids = [*aux["input_ids"][i][:max_len-1], 102]
        aux["input_ids"][i] = aux_input_ids.copy()
      else:
        aux["input_ids"][i] = [*aux["input_ids"][i], *[0]*(max_len - len(aux["input_ids"][i]))]
    for i in range(len(aux["token_type_ids"])):
      if len(aux["token_type_ids"][i]) > max_len:
        aux_token_type_ids = [*aux["token_type_ids"][i][:max_len-1], aux["token_type_ids"][i][-1]]
        aux["token_type_ids"][i] = aux_token_type_ids.copy()
      else:
        aux["token_type_ids"][i] = [*aux["token_type_ids"][i], *[0]*(max_len - len(aux["token_type_ids"][i]))]
    for i in range(len(aux["attention_mask"])):
      if len(aux["attention_mask"][i]) > max_len:
        aux_attention_mask = [*aux["attention_mask"][i][:max_len-1], aux["attention_mask"][i][-1]]
        aux["attention_mask"][i] = aux_attention_mask.copy()
      else:
        aux["attention_mask"][i] = [*aux["attention_mask"][i], *[0]*(max_len - len(aux["attention_mask"][i]))]

  elif method is "ht":
    print('methond selected was: head + tail')
    head_len = int(max_len * 0.25)
    tail_len = int(max_len * 0.75)
    for i in range(len(aux["input_ids"])):
      if len(aux["input_ids"][i]) > max_len:
        aux_input_ids = [*aux["input_ids"][i][:head_len], *aux["input_ids"][i][-tail_len:]]
        aux["input_ids"][i] = aux_input_ids.copy()
      else:
        aux["input_ids"][i] = [*aux["input_ids"][i], *[0]*(max_len - len(aux["input_ids"][i]))]
    for i in range(len(aux["token_type_ids"])):
      if len(aux["token_type_ids"][i]) > max_len:
        aux_token_type_ids = [*aux["token_type_ids"][i][:head_len], *aux["token_type_ids"][i][-tail_len:]]
        aux["token_type_ids"][i] = aux_token_type_ids.copy()
      else:
        aux["token_type_ids"][i] = [*aux["token_type_ids"][i], *[0]*(max_len - len(aux["token_type_ids"][i]))]
    for i in range(len(aux["attention_mask"])):
      if len(aux["attention_mask"][i]) > max_len:
        aux_attention_mask = [*aux["attention_mask"][i][:head_len], *aux["attention_mask"][i][-tail_len:]]
        aux["attention_mask"][i] = aux_attention_mask.copy()
      else:
        aux["attention_mask"][i] = [*aux["attention_mask"][i], *[0]*(max_len - len(aux["attention_mask"][i]))]

  return aux

In [ ]:
train_encodings = truncate(train_encodings, method="t", max_len=128)
val_encodings = truncate(val_encodings, method="t", max_len=512)
test_encodings = truncate(test_encodings, method="t", max_len=512)

methond selected was: head + tail
methond selected was: head + tail
methond selected was: head + tail


In [ ]:
import torch

class FNNDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = FNNDataset(train_encodings, train_labels)
val_dataset = FNNDataset(val_encodings, val_labels)
test_dataset = FNNDataset(test_encodings, test_labels)

In [ ]:
# Checks if there is a gpu available
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU: {}'.format(torch.cuda.get_device_name(0)))
else:
    print('GPU device not found')

Found GPU: Tesla T4


In [ ]:
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from datasets import load_metric

def compute_metrics(model, eval_dataloader, metric):
  model.eval()
  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

  return metric.compute()

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)
test_dataloader = DataLoader(test_dataset, batch_size=16)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(16):
  model.train()
  for batch in train_loader:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs[0]
    loss.backward()
    optim.step()
  print('eval accuracy results for epoch ' + str(epoch) + ': ' + str(compute_metrics(model, val_dataloader, load_metric('accuracy'))['accuracy']))
  print('')

model.eval()
print('trainig has finished')
print('-------------------------------------------------------------------')
print('final evaluation is: ')
whantedStatistics = ['accuracy', 'f1', 'precision', 'recall']
for statistic in whantedStatistics:
  metric = load_metric(statistic)
  print(statistic + ': ' + str(compute_metrics(model, test_dataloader, metric)[statistic]))

eval accuracy results for epoch 0: 0.8224799286351472

eval accuracy results for epoch 1: 0.848349687778769

eval accuracy results for epoch 2: 0.83942908117752

eval accuracy results for epoch 3: 0.8331846565566459

eval accuracy results for epoch 4: 0.8385370205173952

eval accuracy results for epoch 5: 0.8421052631578947

eval accuracy results for epoch 6: 0.8421052631578947

eval accuracy results for epoch 7: 0.8331846565566459

eval accuracy results for epoch 8: 0.8287243532560215

eval accuracy results for epoch 9: 0.8385370205173952

eval accuracy results for epoch 10: 0.8447814451382694

eval accuracy results for epoch 11: 0.8322925958965209

eval accuracy results for epoch 12: 0.8349687778768956

eval accuracy results for epoch 13: 0.8376449598572703

eval accuracy results for epoch 14: 0.8412132024977699

eval accuracy results for epoch 15: 0.8429973238180196

trainig has finished
-------------------------------------------------------------------
final evaluation is: 
accura